In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas, shapely
from pymongo import MongoClient,GEOSPHERE

In [2]:
def geo_frame(df):
    df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.latitude, df.longitude))
    df['geometry']=df['geometry'].apply(lambda x:shapely.geometry.mapping(x))
    return df

### Trying geo queries with differente collections

In [32]:
data = pd.read_csv("data\\test.csv")

In [35]:
data = data = geo_frame(data)

c:\users\juanp\anaconda3\envs\ironhack\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [5]:
# Set connection with MongoDB
conn = MongoClient("localhost:27017")
db = conn.get_database("final_project")
p = db.get_collection("places")
h = db.get_collection("homes")

In [43]:
def lugar(coords, categ):
    return  list(p.find(
        {"category":categ,"geometry": {"$near": {
            "$geometry": {"type": "Point" ,
             "coordinates": coords
       }, "$maxDistance": 500}}},{"name":1, "longitude":1, "latitude":1, "category":1}
    ))
    

In [124]:
def sum_cat(coords):
    result = coords,len(lugar(coords, "gym"))
    if result[1] > 0:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [74]:
def sum_gym(coords):
    result = coords,len(lugar(coords,"gym"))
    if result[1] > 7:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [46]:
def sum_superm(coords):
    result = coords,len(lugar(coords, "supermarket"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [47]:
def sum_medical(coords):
    result = coords,len(lugar(coords, "medical_centre"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [48]:
def sum_nightlife(coords):
    result = coords,len(lugar(coords, "nightlife"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [49]:
def sum_transport(coords):
    result = coords,len(lugar(coords, "transport"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [50]:
def sum_ent(coords):
    result = coords,len(lugar(coords, "general_entertainment"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [51]:
def sum_store(coords):
    result = coords,len(lugar(coords, "clothing_stores"))
    if result[1] > 5:
        r = h.find_one({"latitude":result[0][0],"longitude":result[0][1]},{"_id":0,"province":0, "municipality":0, "showAdress":0, "geometry":0})
        return r
    else:
        pass

In [126]:
def category(choice):

    if choice == "Supermarkets":
        return list(filter(None, map(sum_gym, coordinates_prueba)))
    
    elif choice == "Gyms":
        return  list(filter(None, map(sum_superm, coordinates_prueba)))
    
    elif choice == "Clothing store":
        return list(filter(None, map(sum_store, coordinates_prueba)))
    
    elif choice == "Nightlife":
        return  list(filter(None, map(sum_nightlife, coordinates_prueba)))
    
    elif choice == "Transport":
        return list(filter(None, map(sum_transport, coordinates_prueba)))
    
    elif choice == "Entertainment":
        return  list(filter(None, map(sum_ent, coordinates_prueba)))
    
    elif choice == "Medical centre":
        return list(filter(None, map(sum_medical, coordinates_prueba)))

In [179]:
def final(cat_1, cat_2, cat_3):

    lista = []

    lista.append(category(cat_1))
    lista.append(category(cat_2))
    lista.append(category(cat_3))

    x = pd.DataFrame(lista[0])
    y = pd.DataFrame(lista[1])
    z = pd.DataFrame(lista[2])

    return pd.concat([x,y,z]).drop_duplicates(subset = "latitude")